In [2]:
import geopandas as gpd

gdb_path = "../data/raw/ICADisplay.gdb"
feeder = gpd.read_file(gdb_path, driver='fileGDB', layer="FeederDetail")
# feeder_load = gpd.read_file(gdb_path, driver='fileGDB', layer="FeederLoadProfile")
line = gpd.read_file(gdb_path, driver='fileGDB', layer="LineDetail")
substations = gpd.read_file(gdb_path, driver='fileGDB', layer="Substations")
# substation_load = gpd.read_file(gdb_path, driver='fileGDB', layer="SubstationLoadProfile")

In [4]:
# Convert to GeoJSON and save to file
feeder = feeder.to_crs(epsg=4326)
feeder.to_file("../data/geojson/feeder.geojson", driver='GeoJSON')

In [10]:
# Feeder less than 100 kW
import pandas as pd

aggregated_capacity = line.groupby('FeederId').agg({'LoadCapacity_kW': 'sum'}).reset_index()
feeder_capacity = pd.merge(feeder, aggregated_capacity, left_on='FeederID', right_on='FeederId', how='inner')

capacity = 100 # 100 kW = low
low_capacity_feeders = feeder_capacity[feeder_capacity['LoadCapacity_kW'] < capacity]

low_capacity_lines = pd.merge(line, low_capacity_feeders[['FeederID']], left_on='FeederId', right_on='FeederID', how='inner')
low_capacity_lines_gdf = gpd.GeoDataFrame(low_capacity_lines, geometry='geometry')

low_capacity_lines_gdf.to_file("../data/geojson/low_capacity_feeders.geojson", driver='GeoJSON')

In [11]:
line = line.to_crs(epsg=4326)
line.to_file("../data/geojson/line.geojson", driver='GeoJSON')

In [12]:
substations = substations.to_crs(epsg=4326)
substations.to_file("../data/geojson/substations.geojson", driver='GeoJSON')
# substation_load.to_file("../data/geojson/substation_load.geojson", driver='GeoJSON')

In [1]:
import geopandas as gpd
import pandas as pd

# Paths to the data
gdb_path = "../data/raw/ICADisplay.gdb"

# Reading the data
feeder = gpd.read_file(gdb_path, driver='fileGDB', layer="FeederDetail")
line = gpd.read_file(gdb_path, driver='fileGDB', layer="LineDetail")

# Converting to the correct coordinate system
feeder = feeder.to_crs(epsg=4326)
line = line.to_crs(epsg=4326)

In [2]:
# Aggregating load capacity by FeederId
aggregated_capacity = line.groupby('FeederId').agg({'LoadCapacity_kW': 'sum'}).reset_index()

# Merging feeder data with aggregated capacity
feeder_capacity = pd.merge(feeder, aggregated_capacity, left_on='FeederID', right_on='FeederId', how='inner')

# Check columns after merge
print("Feeder Capacity Columns:", feeder_capacity.columns)

# Filtering for MinElectrification
min_electrification = feeder_capacity[feeder_capacity['ResCust'] * 20.3 <= feeder_capacity['LoadCapacity_kW']]

min_electrification.head()

# Filtering for MidElectrification
mid_electrification = feeder_capacity[feeder_capacity['ResCust'] * 38 <= feeder_capacity['LoadCapacity_kW']]

# # Filtering for MaxElectrification
max_electrification = feeder_capacity[feeder_capacity['ResCust'] * 50 <= feeder_capacity['LoadCapacity_kW']]

Feeder Capacity Columns: Index(['Substation', 'FeederID', 'Feeder_Name', 'Nominal_Voltage',
       'Load_Profile_Redaction', 'ResCust', 'ComCust', 'IndCust', 'AgrCust',
       'OthCust', 'Existing_DG', 'Queued_DG', 'Total_DG', 'Show', 'VOLTNUM',
       'Shape_Length', 'geometry', 'FeederId', 'LoadCapacity_kW'],
      dtype='object')


In [12]:
min_electrification.head()

,Substation,FeederID,Feeder_Name,Nominal_Voltage,Load_Profile_Redaction,ResCust,ComCust,IndCust,AgrCust,OthCust,Existing_DG,Queued_DG,Total_DG,Show,VOLTNUM,Shape_Length,geometry,FeederId,LoadCapacity_kW
0,VACA DIXON,063591102,VACA DIXON 1102,12kV,Yes,1433,101,24,0,6,8280,5770,14050,1,12.00,26933.320905,"MULTILINESTRING ((-121.94347 38.39530, -121.94...",063591102,356430
1,OAKLAND C,012011114,OAKLAND C 1114,12kV,No,2242,274,138,3,20,1320,590,1910,1,12.00,11458.048102,"MULTILINESTRING ((-122.27050 37.79559, -122.27...",012011114,850680
2,SAUSALITO,042490402,SAUSALITO 0402,4.16kV,Yes,523,8,1,0,0,360,160,520,1,4.16,5054.876335,"MULTILINESTRING ((-122.49615 37.85822, -122.49...",042490402,30160
3,BUENA VISTA,182261106,BUENA VISTA 1106,12kV,Yes,14,25,18,49,1,200,200,400,1,12.00,23941.399005,"MULTILINESTRING ((-121.61371 36.64274, -121.61...",182261106,506850
5,VACA DIXON,063591105,VACA DIXON 1105,12kV,Yes,862,81,19,11,3,6930,1990,8920,1,12.00,84083.835282,"MULTILINESTRING ((-121.93618 38.41771, -121.93...",063591105,360050


In [ ]:
mid_electrification.head()

In [14]:
mid_electrification.head()

,Substation,FeederID,Feeder_Name,Nominal_Voltage,Load_Profile_Redaction,ResCust,ComCust,IndCust,AgrCust,OthCust,Existing_DG,Queued_DG,Total_DG,Show,VOLTNUM,Shape_Length,geometry,FeederId,LoadCapacity_kW
0,VACA DIXON,063591102,VACA DIXON 1102,12kV,Yes,1433,101,24,0,6,8280,5770,14050,1,12.00,26933.320905,"MULTILINESTRING ((-121.94347 38.39530, -121.94...",063591102,356430
1,OAKLAND C,012011114,OAKLAND C 1114,12kV,No,2242,274,138,3,20,1320,590,1910,1,12.00,11458.048102,"MULTILINESTRING ((-122.27050 37.79559, -122.27...",012011114,850680
2,SAUSALITO,042490402,SAUSALITO 0402,4.16kV,Yes,523,8,1,0,0,360,160,520,1,4.16,5054.876335,"MULTILINESTRING ((-122.49615 37.85822, -122.49...",042490402,30160
3,BUENA VISTA,182261106,BUENA VISTA 1106,12kV,Yes,14,25,18,49,1,200,200,400,1,12.00,23941.399005,"MULTILINESTRING ((-121.61371 36.64274, -121.61...",182261106,506850
5,VACA DIXON,063591105,VACA DIXON 1105,12kV,Yes,862,81,19,11,3,6930,1990,8920,1,12.00,84083.835282,"MULTILINESTRING ((-121.93618 38.41771, -121.93...",063591105,360050


In [18]:
max_electrification.head()

,Substation,FeederID,Feeder_Name,Nominal_Voltage,Load_Profile_Redaction,ResCust,ComCust,IndCust,AgrCust,OthCust,Existing_DG,Queued_DG,Total_DG,Show,VOLTNUM,Shape_Length,geometry,FeederId,LoadCapacity_kW
0,VACA DIXON,063591102,VACA DIXON 1102,12kV,Yes,1433,101,24,0,6,8280,5770,14050,1,12.00,26933.320905,"MULTILINESTRING ((-121.94347 38.39530, -121.94...",063591102,356430
1,OAKLAND C,012011114,OAKLAND C 1114,12kV,No,2242,274,138,3,20,1320,590,1910,1,12.00,11458.048102,"MULTILINESTRING ((-122.27050 37.79559, -122.27...",012011114,850680
2,SAUSALITO,042490402,SAUSALITO 0402,4.16kV,Yes,523,8,1,0,0,360,160,520,1,4.16,5054.876335,"MULTILINESTRING ((-122.49615 37.85822, -122.49...",042490402,30160
3,BUENA VISTA,182261106,BUENA VISTA 1106,12kV,Yes,14,25,18,49,1,200,200,400,1,12.00,23941.399005,"MULTILINESTRING ((-121.61371 36.64274, -121.61...",182261106,506850
5,VACA DIXON,063591105,VACA DIXON 1105,12kV,Yes,862,81,19,11,3,6930,1990,8920,1,12.00,84083.835282,"MULTILINESTRING ((-121.93618 38.41771, -121.93...",063591105,360050


In [15]:
# Function to create GeoDataFrame and save to GeoJSON
def create_geojson(df, line, filename):
    filtered_lines = pd.merge(line, df[['FeederID']], left_on='FeederId', right_on='FeederID', how='inner')
    print(f"Columns after first merge for {filename}:", filtered_lines.columns)
    filtered_lines = pd.merge(filtered_lines, df[['FeederID', 'Feeder_Name', 'LoadCapacity_kW', 'ResCust', 'ComCust', 'IndCust']], on='FeederID', how='left')  # Merge additional metadata
    print(f"Columns after second merge for {filename}:", filtered_lines.columns)
    filtered_lines_gdf = gpd.GeoDataFrame(filtered_lines, geometry='geometry')
    filtered_lines_gdf = filtered_lines_gdf.rename(columns={
        'LoadCapacity_kW_y': 'LoadCapacity_kW'
    })
    filtered_lines_gdf = filtered_lines_gdf[[
        'Feeder_Name', 'FeederID', 'LoadCapacity_kW', 'ResCust', 'ComCust', 'IndCust', 'geometry'
    ]]
    filtered_lines_gdf.to_crs(epsg=4326).to_file(f"../data/geojson/{filename}.geojson", driver='GeoJSON')

# Create GeoJSON files for each electrification level
create_geojson(min_electrification, line, "min_electrification")
create_geojson(mid_electrification, line, "mid_electrification")
create_geojson(max_electrification, line, "max_electrification")

Columns after first merge for min_electrification: Index(['FeederId', 'FeederName', 'Globalid', 'CSV_LineSection',
       'ICA_Analysis_Date', 'LoadCapacity_kW', 'GenCapacity_kW',
       'GenericPVCapacity_kW', 'GenCapacity_no_OpFlex_kW',
       'GenericCapacity_no_OpFlex_kW', 'Shape_Length', 'geometry', 'FeederID'],
      dtype='object')
Columns after second merge for min_electrification: Index(['FeederId', 'FeederName', 'Globalid', 'CSV_LineSection',
       'ICA_Analysis_Date', 'LoadCapacity_kW_x', 'GenCapacity_kW',
       'GenericPVCapacity_kW', 'GenCapacity_no_OpFlex_kW',
       'GenericCapacity_no_OpFlex_kW', 'Shape_Length', 'geometry', 'FeederID',
       'Feeder_Name', 'LoadCapacity_kW_y', 'ResCust', 'ComCust', 'IndCust'],
      dtype='object')
Columns after first merge for mid_electrification: Index(['FeederId', 'FeederName', 'Globalid', 'CSV_LineSection',
       'ICA_Analysis_Date', 'LoadCapacity_kW', 'GenCapacity_kW',
       'GenericPVCapacity_kW', 'GenCapacity_no_OpFlex_kW',


In [ ]:
create_geojson(min_electrification, line, "min_electrification")

In [16]:
create_geojson(mid_electrification, line, "mid_electrification")


Columns after first merge for mid_electrification: Index(['FeederId', 'FeederName', 'Globalid', 'CSV_LineSection',
       'ICA_Analysis_Date', 'LoadCapacity_kW', 'GenCapacity_kW',
       'GenericPVCapacity_kW', 'GenCapacity_no_OpFlex_kW',
       'GenericCapacity_no_OpFlex_kW', 'Shape_Length', 'geometry', 'FeederID'],
      dtype='object')
Columns after second merge for mid_electrification: Index(['FeederId', 'FeederName', 'Globalid', 'CSV_LineSection',
       'ICA_Analysis_Date', 'LoadCapacity_kW_x', 'GenCapacity_kW',
       'GenericPVCapacity_kW', 'GenCapacity_no_OpFlex_kW',
       'GenericCapacity_no_OpFlex_kW', 'Shape_Length', 'geometry', 'FeederID',
       'Feeder_Name', 'LoadCapacity_kW_y', 'ResCust', 'ComCust', 'IndCust'],
      dtype='object')


In [17]:
create_geojson(max_electrification, line, "max_electrification")

Columns after first merge for max_electrification: Index(['FeederId', 'FeederName', 'Globalid', 'CSV_LineSection',
       'ICA_Analysis_Date', 'LoadCapacity_kW', 'GenCapacity_kW',
       'GenericPVCapacity_kW', 'GenCapacity_no_OpFlex_kW',
       'GenericCapacity_no_OpFlex_kW', 'Shape_Length', 'geometry', 'FeederID'],
      dtype='object')
Columns after second merge for max_electrification: Index(['FeederId', 'FeederName', 'Globalid', 'CSV_LineSection',
       'ICA_Analysis_Date', 'LoadCapacity_kW_x', 'GenCapacity_kW',
       'GenericPVCapacity_kW', 'GenCapacity_no_OpFlex_kW',
       'GenericCapacity_no_OpFlex_kW', 'Shape_Length', 'geometry', 'FeederID',
       'Feeder_Name', 'LoadCapacity_kW_y', 'ResCust', 'ComCust', 'IndCust'],
      dtype='object')
